# Day 17

第二次出现康威生命游戏变体，不过是多维度宇宙中的生命游戏。

# Part I

下面的代码是在完成第二部分的时候重构的代码，因为第一部分是三维空间的，而第二部分是思维空间的，为了避免复制粘贴代码的尴尬，所以将Dimension类重构为可以支撑n个维度的通用代码。

首先是单元格状态的枚举量：

In [1]:
from enum import Enum

State = Enum('State', ('active', 'inactive'))

核心逻辑类Dimemsions，首先将将输入的二维数据转换成n维的空间坐标，然后存储在一个字典cells中。然后记录目前坐标每个维度的最小值和最大值，Dimensions.gen属性记录的是n维中相邻单元格坐标的差值的列表，每个差值都是n维的元组：

In [2]:
from typing import Generator
from itertools import product
from numba import jit

class Dimensions(object):
    '''
    生命游戏宇宙类
    '''
    def __init__(self, dims: int, data: str):
        '''
        dims 维度个数
        data 二维输入数据的字符串，包括换行符
        '''
        lines = data.split('\n')
        self.dims = dims
        # 初始化各维度最小值，全部为0
        self.mins = [0] * dims
        # 初始化各维度最大值，前两个维度为输入矩阵的形状，后面的维度都为0
        self.maxs = [len(lines), len(lines)] + [0] * (dims - 2)
        self.cells = {}
        # 预生成相邻单元格的坐标差值列表，每个相邻单元格以一个元组表示，例如，三维(-1, -1, 0)
        self.gen = [diff for diff in product(*[range(-1, 2) for _ in range(self.dims)]) 
                    if any(y != 0 for y in diff)]
        # 生成cells字典，仅记录活着的单元格
        for x, line in enumerate(lines):
            for y, c in enumerate(line.rstrip()):
                if c == '#':
                    self.cells[tuple([x, y] + [0] * (dims - 2))] = State.active
    
    def set_mins_and_maxs(self, *args):
        '''设置各个维度最小值和最大值'''
        if len(args) != self.dims:
            raise ValueError('wrong dimensions')
        for i in range(self.dims):
            if args[i] < self.mins[i]:
                self.mins[i] = args[i]
            if args[i] > self.maxs[i]:
                self.maxs[i] = args[i]

    def count_active_neighbors(self, *args) -> int:
        '''计算活着的邻居的个数'''
        if len(args) != self.dims:
            raise ValueError('wrong dimensions')
        counter = 0
        for diff in self.gen:
            if not all(self.mins[i] <= args[i] + diff[i] <= self.maxs[i] for i in range(self.dims)):
                continue
            coords = [args[i] + diff[i] for i in range(self.dims)]
            state = self.cells.get(tuple(coords), State.inactive)
            if state == State.active:
                counter += 1
        return counter
    
    def next_cycle(self):
        '''游戏下一轮状态，使用一个新的字典记录，然后整体替换原来的cells'''
        new_cells = {}
        # 使用product来得到整个宇宙目前可能存在声明的单元格坐标
        g = product(*[range(self.mins[i] - 1, self.maxs[i] + 2) for i in range(self.dims)])
        # 依据规则生成一轮新的宇宙
        for coords in g:
            neighbors = self.count_active_neighbors(*coords)
            state = self.cells.get(coords, State.inactive)
            if state == State.active and (neighbors == 2 or neighbors == 3):
                new_cells[coords] = State.active
                self.set_mins_and_maxs(*coords)
            if state == State.inactive and neighbors == 3:
                new_cells[coords] = State.active
                self.set_mins_and_maxs(*coords)
        self.cells = new_cells
    
    def count_all_active(self) -> int:
        '''计算所有生命数量'''
        return len(self.cells)

# Part II

经过重构后，第一部分和第二部分的逻辑可以整合在一起，因为两个部分的差别仅仅就是维度数量：

In [3]:
def all_solution(input_file: str, dims: int, cycles: int=6) -> int:
    with open(input_file) as fn:
        universe = Dimensions(dims, fn.read())
    for i in range(cycles):
        universe.next_cycle()
    return universe.count_all_active()

单元测试：

In [4]:
assert(all_solution('testcase1.txt', 3) == 112)
assert(all_solution('testcase1.txt', 4) == 848)

第一部分结果：

In [5]:
all_solution('input.txt', 3)

401

第二部分结果，随着维度增大，计算复杂度急剧增大，暂未有好办法优化Python的性能：

In [6]:
all_solution('input.txt', 4)

2224